In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
import streamlit as st

# ---------------------------------------------
# Adaptación de Estilo para Streamlit
# ---------------------------------------------
DEFAULT_FONT = "Aptos Light, Arial, sans-serif"
# Orden de colores por defecto: [Barra, Acento/Anotación]
DEFAULT_PALETTE = ["#889064", "#ff9f18"] # Verde corporativo (Barra), Naranja (Acento)

# Obtener variables inyectadas de Streamlit (si existen) o usar valores por defecto
PALETTE = globals().get('active_palette', DEFAULT_PALETTE)
FONT = globals().get('active_font', DEFAULT_FONT)

# Asignar variables de estilo
COLOR_BARRA = PALETTE[0] if len(PALETTE) > 0 else DEFAULT_PALETTE[0]
COLOR_ACCENT = PALETTE[1] if len(PALETTE) > 1 else DEFAULT_PALETTE[1]
FONT_FAMILY = FONT

# =============================
# CONFIGURACIÓN
# =============================
TOKEN = "7c7245244cd2df18b2b03e0834258450b2ab7c578910115fb8975a7f1c48b9e8"
SERIE_ID = "SP1"  # INPC general
API_URL = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{SERIE_ID}/datos"

# =============================
# OBTENER DATOS DE BANXICO
# =============================
try:
    response = requests.get(API_URL, headers={"Bmx-Token": TOKEN}, timeout=10)
    response.raise_for_status()
    data = response.json()

    # Extraer datos de la serie
    serie = data['bmx']['series'][0]['datos']
    df = pd.DataFrame(serie)
    df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
    df['dato'] = pd.to_numeric(df['dato'])

except requests.exceptions.RequestException as e:
    st.error(f"Error al conectar con la API de Banxico: {e}")
    st.stop()
except KeyError:
    st.error("Error al parsear la respuesta de la API. Verifica el TOKEN y SERIE_ID.")
    st.stop()


# =============================
# CALCULAR INFLACIÓN ANUAL (septiembre-septiembre)
# =============================
df = df.sort_values('fecha')
df.set_index('fecha', inplace=True)

# Filtrar desde 2015
start_date = datetime(2015, 1, 1)
df = df[df.index >= start_date]

inflacion_anual = []
labels = []
years = sorted(set(df.index.year))

for i in range(len(years)-1):
    y1, y2 = years[i], years[i+1]
    # Usamos .tail(1) para asegurar que obtenemos el dato más reciente de septiembre
    sep_y1 = df[(df.index.year == y1) & (df.index.month == 9)].tail(1)
    sep_y2 = df[(df.index.year == y2) & (df.index.month == 9)].tail(1)

    if not sep_y1.empty and not sep_y2.empty:
        val_y1 = sep_y1.iloc[0]['dato']
        val_y2 = sep_y2.iloc[0]['dato']
        tasa = ((val_y2 / val_y1) - 1) * 100
        inflacion_anual.append(round(tasa, 2))
        labels.append(f"{y1}-{y2}")

# =============================
# IDENTIFICAR MÍNIMO Y MÁXIMO HISTÓRICO
# =============================
if inflacion_anual:
    min_val = min(inflacion_anual)
    max_val = max(inflacion_anual)
    min_idx = inflacion_anual.index(min_val)
    max_idx = inflacion_anual.index(max_val)
else:
    st.warning("No hay suficientes datos para calcular la inflación anual.")
    st.stop()


# =============================
# CREAR GRÁFICA
# =============================
fig = go.Figure()

fig.add_trace(go.Bar(
    x=labels,
    y=inflacion_anual,
    marker_color=COLOR_BARRA,
    text=[f"{v:.2f}%" for v in inflacion_anual],
    textposition="outside"
))

# --- NUEVO: Anotación de Fuente de Datos ---
fig.add_annotation(
    text="Fuente: Banco de México (Banxico) / SIE API",
    xref="paper", yref="paper",
    x=0, y=-0.22,  # Debajo del eje X
    showarrow=False,
    xanchor='left',
    yanchor='top',
    font=dict(size=10, color="gray", family=FONT_FAMILY)
)

# Anotaciones Mínimo y Máximo
fig.add_annotation(
    x=labels[min_idx], y=min_val,
    text=f"Mínimo Histórico<br>{labels[min_idx]} – {min_val:.2f}%",
    showarrow=True, arrowhead=2, ax=-80, ay=-40,
    font=dict(family=FONT_FAMILY, color=COLOR_ACCENT),
    bordercolor=COLOR_ACCENT, borderwidth=1, bgcolor="white"
)

fig.add_annotation(
    x=labels[max_idx], y=max_val,
    text=f"Máximo Histórico<br>{labels[max_idx]} – {max_val:.2f}%",
    showarrow=True, arrowhead=2, ax=80, ay=-40,
    font=dict(family=FONT_FAMILY, color=COLOR_ACCENT),
    bordercolor=COLOR_ACCENT, borderwidth=1, bgcolor="white"
)

fig.update_layout(
    # --- MODIFICADO: Título centrado explícito ---
    title=dict(
        text="Índice de Inflación al Consumidor (Variación Anual Septiembre-Septiembre)",
        x=0.5,
        xanchor='center',
        font=dict(family=FONT_FAMILY, size=20)
    ),
    font=dict(family=FONT_FAMILY, size=14),
    plot_bgcolor="white",
    # --- MODIFICADO: Aumentamos margen inferior (b=80) para la fuente ---
    margin=dict(l=40, r=40, t=80, b=80),
    yaxis=dict(
        title="Inflación (%)",
        tickformat=".2f%",
        range=[0, max(inflacion_anual) + 1],
        showgrid=True,
        gridcolor="#E1DEE0"
    ),
    xaxis=dict(
        title="Periodo",
        tickangle=0,
        tickmode="array",
        tickvals=labels,
        tickson="boundaries",
        tickfont=dict(
            family=FONT_FAMILY,
            size=12,
            color="black"
        )
    )
)

# -------------------------
# 5) Muestra en Streamlit
# -------------------------
st.plotly_chart(fig, use_container_width=True)